In [ ]:
pip install cvxportfolio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.9/352.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00


In [ ]:
UNIVERSE = ['AAV.BK',
 'ACE.BK',
 'ADVANC.BK',
 'AMATA.BK',
 'AOT.BK',
 'AP.BK',
 'BANPU.BK',
 'BBL.BK',
 'BCH.BK',
 'BCP.BK',
 'BCPG.BK',
 'BDMS.BK',
 'BEM.BK',
 'BGRIM.BK',
 'BH.BK',
 'BJC.BK',
 'BLA.BK',
 'BSET100.BK',
 'BTS.BK',
 'CBG.BK',
 'CENTEL.BK',
 'CHG.BK',
 'CK.BK',
 'CKP.BK',
 'COM7.BK',
 'CPALL.BK',
 'CPF.BK',
 'CPN.BK',
 'EA.BK',
 'ERW.BK',
 'GLOBAL.BK',
 'GPSC.BK',
 'GULF.BK',
 'GUNKUL.BK',
 'HANA.BK',
 'HMPRO.BK',
 'INTUCH.BK',
 'IRPC.BK',
 'IVL.BK',
 'JAS.BK',
 'JMT.BK',
 'KBANK.BK',
 'KCE.BK',
 'KKP.BK',
 'KTB.BK',
 'KTC.BK',
 'LH.BK',
 'MEGA.BK',
 'MINT.BK',
 'MTC.BK',
 'PLANB.BK',
 'PRM.BK',
 'PTT.BK',
 'PTTEP.BK',
 'PTTGC.BK',
 'RATCH.BK',
 'RCL.BK',
 'SAWAD.BK',
 'SCC.BK',
 'SJWD.BK',
 'SPALI.BK',
 'SPRC.BK',
 'STA.BK',
 'TASCO.BK',
 'TCAP.BK',
 'TISCO.BK',
 'TOA.BK',
 'TOP.BK',
 'TRUE.BK',
 'TU.BK',
 'WHA.BK']

In [ ]:
# UNIVERSE = list(set(SET).intersection(set(SET100)))
len(UNIVERSE)

71

In [ ]:
import cvxportfolio as cvx
simulator = cvx.StockMarketSimulator(UNIVERSE)

Updating data............

....

.......................................................


In [ ]:
simulator_test = cvx.StockMarketSimulator(['BSET100.BK'])
benchmark = simulator_test.backtest(cvx.Uniform(), start_time='2019-01-02', end_time='2024-08-01')

Updating data.


single factor index forecaster


In [ ]:
import numpy as np
class CustomForecaster: # pylint: disable=too-few-public-methods
    """Expected return as mean of recent window of past returns.

    This is only meant as an example of how to define a custom forecaster;
    it is not very interesting. Since version ``1.2.0`` a similar
    functionality has been included in the default forecasters classes.

    :param window: Window used for the mean returns.
    :type window: int
    """

    def __init__(self, window=20):
        self.window = window

    def CAPM_regression(self, price_df):

      price_df_temp = price_df.iloc[-self.window:,:-1]
      index_return = price_df['BSET100.BK'].iloc[-self.window:,]
      n_columns = price_df_temp.shape[1] # Number of columns in the DataFrame except dollar value
      alpha = np.zeros(n_columns)
      beta = np.zeros(n_columns)
      for i, column in enumerate(price_df_temp.columns):
        stock_return = np.array(price_df_temp[column])
        alpha[i], beta[i] =  np.polyfit(index_return, stock_return, 1)

      return alpha, beta

    def values_in_time(self, past_returns, **kwargs):
      alpha, beta = self.CAPM_regression(past_returns)
      return alpha + beta * (past_returns.iloc[-self.window:,:-1].mean())

costom_forecaster = CustomForecaster(252)

market_neutral

In [ ]:
import numpy as np
import cvxportfolio as cvx
target_volatility = 0.1 / np.sqrt(252) # annual std

risk_model = cvx.FactorModelCovariance(num_factors = 10)

constraints = [
    risk_model <= target_volatility**2,
    cvx.DollarNeutral(),
    cvx.MarketNeutral(),
    cvx.LeverageLimit(7),
    cvx.MaxWeights(0.1),
    cvx.MinWeights(-0.1),
]

policies = []
for days in [10,30,60,180,252]:
  policies.append(cvx.MultiPeriodOptimization(
      cvx.ReturnsForecast(r_hat = CustomForecaster(days))-cvx.StocksTransactionCost(),
      constraints = constraints,
      ignore_dpp=True,
      solver='ECOS',
      planning_horizon=5))

sim = simulator

result = sim.backtest_many(policies, start_time='2022-01-02', end_time='2024-07-30')

print('RESULT:')
print(result)

RESULT:
[
#################################################################
Universe size                                                  71
Initial timestamp                       2022-01-04 03:00:00+00:00
Final timestamp                         2024-07-26 03:00:00+00:00
Number of periods                                             622
Initial value (USDOLLAR)                                1.000e+06
Final value (USDOLLAR)                                  6.589e+05
Profit (USDOLLAR)                                      -3.411e+05
                                                                 
Avg. return (annualized)                                   -15.7%
Volatility (annualized)                                     11.5%
Avg. excess return (annualized)                            -19.3%
Avg. active return (annualized)                            -19.3%
Excess volatility (annualized)                              11.5%
Active volatility (annualized)                              11.5%
